# Using CountVectorizer to compute a similarity score
***Matt Paterson***<br>
***Machine Learning Engineer***<br>
***Santa Cruz, California***<br>
***10/16/2021***<br>

In this challenge, I'll take four csv's with raw data about users of a platform who view videos, review the videos, and also voluntarily submit their interests in the videos, as well as the videos and video authors. 

I will create a model and api that allows an administrator/user (comapny) to input a user_handle (customer) from the existing group of user_handle(s) and get an output of the closest users to the input user_handle.

For the Quick and Dirty model, 
- I will use a simple cosine similarity score for the users, 
- run a dbscan model as an additional input column,
- employ some Natural Language Processing techniques to find 
    - similar course tags and 
    - interest tags and 
    - assessment tags where they are inconsistent, and will 
- utilize scikit-learn's OneHotEncoder to quickly vectorize categorical data

I will then create a lookup table in DynamoDB that can store the resulting users table, allowing a RESTful API to query the database through Amazon API Gateway through the depoloyment of an Amazon SageMaker Model Endpoint.

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

bucket='s3://mp-pluralsight-demo-20211016/'

from sklearn.feature_extraction.text import CountVectorizer
# Before final push, add any imports that come later in this program to this top box

## Load `course_tags.csv`, `user_assessment_scores.csv`, `user_course_views.csv`, and `user_interests.csv`
---

In [10]:
courses = pd.read_csv(bucket +'data/course_tags.csv')
print("courses.shape is", courses.shape)
courses.head()

courses.shape is (11337, 2)


,course_id,course_tags
0,12-principles-animation-toon-boom-harmony-1475,2d-animation
1,2d-racing-game-series-unity-5-1312,game-design
2,2d-racing-games-unity-volume-2-1286,game-art
3,2d-racing-games-unity-volume-2-1286,digital-painting
4,2d-racing-games-unity-volume-2-1286,image-editing


In [11]:
courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11337 entries, 0 to 11336
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course_id    11337 non-null  object
 1   course_tags  11225 non-null  object
dtypes: object(2)
memory usage: 177.3+ KB


In [12]:
courses=courses.dropna()

In [17]:
vec = CountVectorizer()

#tokenize
vec.fit(courses.course_id)

CountVectorizer()

In [18]:
vec.vocabulary_

{'12': 182,
 'principles': 4776,
 'animation': 2239,
 'toon': 5664,
 'boom': 2480,
 'harmony': 3612,
 '1475': 431,
 '2d': 1465,
 'racing': 4907,
 'game': 3485,
 'series': 5196,
 'unity': 5771,
 '1312': 286,
 'games': 3487,
 'volume': 5876,
 '1286': 261,
 '1311': 285,
 '3d': 1601,
 'coat': 2704,
 'character': 2631,
 'concept': 2774,
 'sculpting': 5164,
 'techniques': 5564,
 '2526': 1400,
 'getting': 3513,
 'started': 5399,
 '2487': 1370,
 'integration': 3797,
 'relighting': 4986,
 'nuke': 4437,
 '1157': 144,
 'models': 4280,
 'presenting': 4764,
 'online': 4473,
 '2385': 1285,
 'pan': 4556,
 'tile': 5629,
 'sky': 5296,
 '81': 1960,
 'printing': 4780,
 'design': 2993,
 'selective': 5183,
 'laser': 3988,
 'sintering': 5276,
 'rotation': 5084,
 'trackers': 5681,
 'trigonometry': 5714,
 '284': 1450,
 '3ds': 1603,
 'max': 4178,
 '2011': 928,
 'motion': 4314,
 'graphics': 3547,
 '585': 1763,
 '2017': 935,
 'introduction': 3832,
 '2517': 1394,
 'modeling': 4279,
 'cartoon': 2580,
 'props': 483

In [6]:
#courses.groupby('course_id').sum()a
courses[courses['course_id']=='artists-guide-mel-3163']

,course_id,course_tags
983,artists-guide-mel-3163,NaN


In [7]:
courses_wide = pd.get_dummies(courses, columns=['course_id'])

In [8]:
courses_wide.shape

(11337, 5943)

In [9]:
#courses_wide.groupby('')

In [10]:
courses

,course_id,course_tags
0,12-principles-animation-toon-boom-harmony-1475,2d-animation
1,2d-racing-game-series-unity-5-1312,game-design
2,2d-racing-games-unity-volume-2-1286,game-art
3,2d-racing-games-unity-volume-2-1286,digital-painting
4,2d-racing-games-unity-volume-2-1286,image-editing
...,...,...
11332,zombie-photo-manipulation-techniques-photoshop...,image-editing
11333,z-os-mainframe-introduction,mainframe
11334,z-os-tso-ispf-environment-introduction,mainframe
11335,zsphere-modeling-zbrush-3505,3d-modeling


In [11]:
courses=courses_wide.copy()

In [12]:
course_titles=[]
course_titles=courses.drop(columns='course_tags').columns
len(course_titles)

5942

In [13]:
assess = pd.read_csv(bucket +'data/user_assessment_scores.csv')
print("assess.shape is", assess.shape)
assess.head()

assess.shape is (6571, 4)


,user_handle,assessment_tag,user_assessment_date,user_assessment_score
0,7487,angular-js,2017-08-11 19:03:38,134
1,7487,css,2017-08-11 20:09:56,38
2,7487,html5,2017-07-31 18:59:37,84
3,7487,java,2017-07-31 18:49:27,149
4,7487,javascript,2017-07-31 19:05:03,92


In [14]:
assess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6571 entries, 0 to 6570
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   user_handle            6571 non-null   int64 
 1   assessment_tag         6571 non-null   object
 2   user_assessment_date   6571 non-null   object
 3   user_assessment_score  6571 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 205.5+ KB


In [15]:
assess

,user_handle,assessment_tag,user_assessment_date,user_assessment_score
0,7487,angular-js,2017-08-11 19:03:38,134
1,7487,css,2017-08-11 20:09:56,38
2,7487,html5,2017-07-31 18:59:37,84
3,7487,java,2017-07-31 18:49:27,149
4,7487,javascript,2017-07-31 19:05:03,92
...,...,...,...,...
6566,958,node-js,2017-04-26 20:36:35,245
6567,8887,angular-js,2016-09-30 22:30:48,221
6568,8887,docker,2017-03-24 17:55:06,148
6569,8887,html5,2017-02-10 16:38:53,241


In [16]:
views = pd.read_csv(bucket +'data/user_course_views.csv')
print("views.shape is", views.shape)
views.head()

views.shape is (249238, 6)


,user_handle,view_date,course_id,author_handle,level,view_time_seconds
0,1,2017-06-27,cpt-sp2010-web-designers-branding-intro,875,Beginner,3786
1,1,2017-06-28,cpt-sp2010-web-designers-branding-intro,875,Beginner,1098
2,1,2017-06-28,cpt-sp2010-web-designers-css,875,Intermediate,4406
3,1,2017-07-27,cpt-sp2010-web-designers-css,875,Intermediate,553
4,1,2017-09-12,aws-certified-solutions-architect-professional,281,Advanced,102


In [17]:
views[views['course_id']=='wpf-advanced-topics'].shape

(34, 6)

In [18]:
views.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249238 entries, 0 to 249237
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   user_handle        249238 non-null  int64 
 1   view_date          249238 non-null  object
 2   course_id          249238 non-null  object
 3   author_handle      249238 non-null  int64 
 4   level              249238 non-null  object
 5   view_time_seconds  249238 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 11.4+ MB


In [19]:
#views_dummies=pd.get_dummies(views, columns=['course_id'])

In [20]:
#views_dummies.shape

In [21]:
#views=views_dummies.copy()

In [22]:
interests = pd.read_csv(bucket +'data/user_interests.csv')
print("interests.shape is", interests.shape)
interests.head()

interests.shape is (297526, 3)


,user_handle,interest_tag,date_followed
0,1,mvc-scaffolding,2017-06-27 16:26:52
1,1,mvc2,2017-06-27 16:26:52
2,1,mvc-html-helpers,2017-06-27 16:26:52
3,1,mvc4-ioc,2017-06-27 16:26:52
4,1,mvc-testing,2017-06-27 16:26:52


In [23]:
interests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297526 entries, 0 to 297525
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_handle    297526 non-null  int64 
 1   interest_tag   297526 non-null  object
 2   date_followed  297526 non-null  object
dtypes: int64(1), object(2)
memory usage: 6.8+ MB


## Drop unnecessary columns
---

For this first run through, drop all of the null values from courses as there are a small number of them--112/11337

In [24]:
courses = courses.dropna()

## Merge into a single dataframe
---

Use the user_handle column to merge three of the dataframes and course_id to merge the other

In [25]:
def make_dfs():
    '''
    return updated list with each df for parsing
    '''
    return [courses, assess, views, interests]

In [26]:
dfs=make_dfs()
df_names=['courses', 'assess', 'views', 'interests']

In [27]:
for df in dfs:
    print(df.shape)

(11225, 5943)
(6571, 4)
(249238, 6)
(297526, 3)


In [28]:
for i in range(len(dfs)):
    print("***" + df_names[i] + "***")
    print(dfs[i].dtypes)

***courses***
course_tags                                                      object
course_id_12-principles-animation-toon-boom-harmony-1475          uint8
course_id_2d-racing-game-series-unity-5-1312                      uint8
course_id_2d-racing-games-unity-volume-2-1286                     uint8
course_id_2d-racing-games-unity-volume-4-1311                     uint8
                                                                  ...  
course_id_zend-db-deep-dive                                       uint8
course_id_zmodeler-workflows-2091                                 uint8
course_id_zombie-concept-illustration-photoshop-782               uint8
course_id_zombie-photo-manipulation-techniques-photoshop-1518     uint8
course_id_zsphere-modeling-zbrush-3505                            uint8
Length: 5943, dtype: object
***assess***
user_handle               int64
assessment_tag           object
user_assessment_date     object
user_assessment_score     int64
dtype: object
***views***

In [29]:
dfs[1]['user_assessment_date'].head()

0    2017-08-11 19:03:38
1    2017-08-11 20:09:56
2    2017-07-31 18:59:37
3    2017-07-31 18:49:27
4    2017-07-31 19:05:03
Name: user_assessment_date, dtype: object

In [30]:
assess.head(1)

,user_handle,assessment_tag,user_assessment_date,user_assessment_score
0,7487,angular-js,2017-08-11 19:03:38,134


In [31]:
assess['user_assessment_date'] = pd.to_datetime(assess['user_assessment_date'])

In [32]:
dfs = make_dfs()

In [33]:
for i in range(len(dfs)):
    print("***" + df_names[i] + "***")
    print(dfs[i].dtypes)

***courses***
course_tags                                                      object
course_id_12-principles-animation-toon-boom-harmony-1475          uint8
course_id_2d-racing-game-series-unity-5-1312                      uint8
course_id_2d-racing-games-unity-volume-2-1286                     uint8
course_id_2d-racing-games-unity-volume-4-1311                     uint8
                                                                  ...  
course_id_zend-db-deep-dive                                       uint8
course_id_zmodeler-workflows-2091                                 uint8
course_id_zombie-concept-illustration-photoshop-782               uint8
course_id_zombie-photo-manipulation-techniques-photoshop-1518     uint8
course_id_zsphere-modeling-zbrush-3505                            uint8
Length: 5943, dtype: object
***assess***
user_handle                       int64
assessment_tag                   object
user_assessment_date     datetime64[ns]
user_assessment_score            

How many unique tags exist in each dataframe?

In [34]:
tags = len(courses.course_tags.value_counts())
#ids = len(courses.course_id.value_counts())

print(f"There are {tags} unique course tags")
#print(f"There are {ids} unique course ids")

There are 998 unique course tags


In [35]:
views.columns

Index(['user_handle', 'view_date', 'course_id', 'author_handle', 'level',
       'view_time_seconds'],
      dtype='object')

In [36]:
assess.columns

Index(['user_handle', 'assessment_tag', 'user_assessment_date',
       'user_assessment_score'],
      dtype='object')

In [37]:
assess_tags = len(assess.assessment_tag.value_counts())

print(f"There are {assess_tags} unique assessment tags")


There are 54 unique assessment tags


In [38]:
interest_tags = len(interests.interest_tag.value_counts())

print(f"There are {interest_tags} unique interest tags")


There are 748 unique interest tags


In [39]:
for df in dfs:
    print(df.columns)

Index(['course_tags',
       'course_id_12-principles-animation-toon-boom-harmony-1475',
       'course_id_2d-racing-game-series-unity-5-1312',
       'course_id_2d-racing-games-unity-volume-2-1286',
       'course_id_2d-racing-games-unity-volume-4-1311',
       'course_id_3d-coat-character-concept-sculpting-techniques-2526',
       'course_id_3d-coat-getting-started-2487',
       'course_id_3d-integration-relighting-nuke-1157',
       'course_id_3d-models-presenting-online-2385',
       'course_id_3d-pan-tile-sky-nuke-81',
       ...
       'course_id_zbrush-mudbox-artists-1416',
       'course_id_zbrush-reference-library-3d-brushes-547',
       'course_id_zbrush-sculpting-modular-structures-2537',
       'course_id_zbrush-sculpting-necroknight-2509',
       'course_id_zbrush-tileable-texture-creation-2438',
       'course_id_zend-db-deep-dive', 'course_id_zmodeler-workflows-2091',
       'course_id_zombie-concept-illustration-photoshop-782',
       'course_id_zombie-photo-manipulatio

In [40]:
df_names

['courses', 'assess', 'views', 'interests']

In [41]:
assess.shape

(6571, 4)

In [42]:
views.shape

(249238, 6)

## Create a `user-course` dataframe
**It should have the following columns to start**
- user_handle
- total_users_courses
- course_id
- course_tags
- first_view_date
- total_views
- avg_viewtime
- level
- author_handle
- interest_tags
- user_assessment_score
- user_avg_assess_score

**It should have a compound-index of user_handle_course_id**<br>

***On second thought, will that be helpful or only serve to delay the time to get to the MVP?***

## Create a `users` dataframe
**This should really have the user_handle as an index**


From here, create a DBScan clustering model and use the clusters from it as an additional feature.<br>

Once completed, run the cosine similarity and create a way to score the users.

This will require that we group each dataframe by the above factors

In [43]:
assess_users = len(assess.user_handle.value_counts())
interests_users = len(interests.user_handle.value_counts())
views_users = len(views.user_handle.value_counts())

print(f"There are {assess_users} user_handles in the assess df")
print(f"There are {interests_users} user_handles in the interests df")
print(f"There are {views_users} user_handles in the views df")

There are 3114 user_handles in the assess df
There are 10000 user_handles in the interests df
There are 8760 user_handles in the views df


I'll need to figure out a logic to create this table that has 
1. a unique row for each comination of user and course that the user viewed
2. the assessment and score that he user gave the course
3. linked together by the course tags, assessment tags
4. whether or not course tag or assessment tag match an interest tag from this user

In [44]:
users = pd.merge(left=interests, right=views, how='outer', left_on='user_handle', right_on='user_handle')

In [45]:
users.shape

(9474074, 8)

In [46]:
users.user_handle.value_counts().shape

(10000,)

We use the outer join merge, or otherwise keep all user_handles total, so that we can construct a dataset that includes those that never took a course but only gave their interests

In [47]:
users = pd.merge(left=users, right=assess, how='outer', left_on='user_handle', right_on='user_handle')

In [48]:
users.shape

(21951197, 11)

In [49]:
users.user_handle.value_counts().shape

(10000,)

In [50]:
#users = pd.merge(left=users, right=courses, how='outer', left_on='course_id', right_on='course_id')

In [51]:
users.shape

(21951197, 11)

In [52]:
users_dummied = pd.get_dummies(users, columns=['course_id'], sparse=True)
users_dummied.shape

(21951197, 5952)

In [53]:
users_dummied.dtypes

user_handle                                                                 int64
interest_tag                                                               object
date_followed                                                              object
view_date                                                                  object
author_handle                                                             float64
                                                                       ...       
course_id_zend-db-deep-dive                                      Sparse[uint8, 0]
course_id_zmodeler-workflows-2091                                Sparse[uint8, 0]
course_id_zombie-concept-illustration-photoshop-782              Sparse[uint8, 0]
course_id_zombie-photo-manipulation-techniques-photoshop-1518    Sparse[uint8, 0]
course_id_zsphere-modeling-zbrush-3505                           Sparse[uint8, 0]
Length: 5952, dtype: object

In [54]:
len(users_dummied['interest_tag'].value_counts())

748

In [55]:
users_twice_dummied = pd.get_dummies(users_dummied, columns=['interest_tag'], sparse=True)

In [56]:
users_twice_dummied.shape

(21951197, 6699)

## Convert remaining non-numeric columns

In [57]:
for dtype in users_twice_dummied.dtypes:
    if dtype == 'object':
        print(dtype)

object
object
object
object


In [58]:
users_twice_dummied = pd.get_dummies(users_twice_dummied, columns=['level'])

In [59]:
users_twice_dummied.columns[:6]

Index(['user_handle', 'date_followed', 'view_date', 'author_handle',
       'view_time_seconds', 'assessment_tag'],
      dtype='object')

In [60]:
users_twice_dummied['view_date'] = pd.to_datetime(users_twice_dummied['view_date'])

In [61]:
len(users_twice_dummied.author_handle.value_counts())

1412

In [62]:
users_s = pd.get_dummies(users_twice_dummied, columns=['author_handle'], sparse=True)

In [63]:
users_s.shape

(21951197, 8112)

In [64]:
for dtype in users_twice_dummied.dtypes:
    if dtype == 'object':
        print(dtype)

object
object


In [65]:
users_s.dtypes[:10]

user_handle                                                            int64
date_followed                                                         object
view_date                                                     datetime64[ns]
view_time_seconds                                                    float64
assessment_tag                                                        object
user_assessment_date                                          datetime64[ns]
user_assessment_score                                                float64
course_id_12-principles-animation-toon-boom-harmony-1475    Sparse[uint8, 0]
course_id_2d-racing-game-series-unity-5-1312                Sparse[uint8, 0]
course_id_2d-racing-games-unity-volume-2-1286               Sparse[uint8, 0]
dtype: object

In [66]:
users_s = users_s.drop(columns='assessment_tag')

In [67]:
for dtype in users_s.dtypes:
    if dtype == 'object':
        print(dtype)

object


## not-so-quick but...
So we've gotten our dataset down to a 100% numeric dataset. We've eliminated the tags entirely, which will be detremental to our final goal, but will help us at least get something started.

We've One Hot Encoded all of the course ID's, the author ID's, and the levels, and thus we should be able to somewhat shrink our dataframe by grouping by the usernames...

In [69]:
users_grouped = users_s.groupby(by='user_handle')
users_s.sample()

,user_handle,date_followed,view_date,view_time_seconds,user_assessment_date,user_assessment_score,course_id_12-principles-animation-toon-boom-harmony-1475,course_id_2d-racing-game-series-unity-5-1312,course_id_2d-racing-games-unity-volume-2-1286,course_id_2d-racing-games-unity-volume-4-1311,...,author_handle_1403.0,author_handle_1404.0,author_handle_1405.0,author_handle_1406.0,author_handle_1407.0,author_handle_1408.0,author_handle_1409.0,author_handle_1410.0,author_handle_1411.0,author_handle_1412.0
6936517,1970,2017-07-07 04:42:04,2017-07-20,85.0,2016-08-25 00:47:34,113.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
users_grouped.sum()

In [71]:
#users_grouped.describe().T

In [ ]:
df_final = users_s.pivot_table( index=users_s.user_handle, aggfunc='sum')
df_final.fillna(0, inplace = True)

## Use DBScan to create clusters of users

In [72]:
from sklearn.cluster import DBSCAN

## Use Cosine Similarity Scores to compare the users to one another

## Save the dataframe to csv and consider DynamoDB, SQLite, PostreSQL, and AWS RDS